In [ ]:
#pip install nltk

In [13]:
import nltk
import numpy as np
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

In [19]:
# Definición del corpus, son palabras clave que el chatbot pueda responder
with open(r'archivos\archivos NLP\corpus_servicios_software.txt', 'r', encoding='utf-8', errors='ignore') as f:
    raw = f.read()

In [20]:
# Preprocesamiento del texto
raw = raw.lower()  # Convertir a minúsculas

In [21]:
# Descargar y configurar nltk si es necesario
nltk.download('punkt')  # Instalar módulo punkt si no está instalado (solo ejecutar la primera vez)
nltk.download('wordnet')  # Instalar módulo wordnet si no está instalado (solo ejecutar la primera vez)
nltk.download('stopwords')  # Instalar módulo stopwords si no está instalado (solo ejecutar la primera vez)
nltk.download('punkt_tab') # Instalar módulo punkt_tab si no está instalado (solo ejecutar la primera vez)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dg_su\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dg_su\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dg_su\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dg_su\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Preprocesamiento del texto con NTLK Corpus

In [22]:
sent_tokens = nltk.sent_tokenize(raw)  # Convierte el corpus a una lista de sentencias
word_tokens = nltk.word_tokenize(raw)  # Convierte el corpus a una lista de palabras

In [23]:
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [24]:
# Ejemplo de preprocesamiento de una frase
ejemplo_frase = "Desarrollamos todo tipo de páginas web, desde sitios web corporativos y tiendas en línea hasta blogs y portafolios personales."
print(LemNormalize(ejemplo_frase))

['desarrollamos', 'todo', 'tipo', 'de', 'páginas', 'web', 'desde', 'sitios', 'web', 'corporativos', 'y', 'tiendas', 'en', 'línea', 'hasta', 'blog', 'y', 'portafolios', 'personales']


In [25]:
# Definición de coincidencias manuales
SALUDOS_INPUTS = ("hola", "buenas", "saludos", "qué tal", "hey", "buenos días")
SALUDOS_OUTPUTS = ["Hola", "Hola, ¿Qué tal?", "Hola, ¿Cómo te puedo ayudar?", "Hola, encantado de hablar contigo"]

def saludos(sentence):
    for word in sentence.split():
        if word.lower() in SALUDOS_INPUTS:
            return random.choice(SALUDOS_OUTPUTS)

Preprocesamient del texto + Evaluaar similitud mensaje usuario - corpus

In [26]:
# Función para determinar la similitud del texto insertado y el corpus
def response(user_response):
    robo_response = ''
    sent_tokens.append(user_response)  # Añade la respuesta del usuario al final del corpus
    tfidfvec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words('spanish'))
    tfidf = tfidfvec.fit_transform(sent_tokens)
    # Evaluar similitud de coseno entre mensaje usuario y el corpus
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    
    if(req_tfidf == 0):
        robo_response = robo_response + "Lo siento, no te he entendido. Si no puedo responder a lo que buscas, ponte en contacto con soporte@empresa.com."
    else:
        robo_response = robo_response + sent_tokens[idx]
    
    sent_tokens.pop(-1)  # Eliminar la respuesta del usuario del corpus
    return robo_response

In [27]:
# Ciclo de conversación del chatbot
flag = True
print("Chatbot: ¡Hola! Soy tu asistente virtual. ¿En qué puedo ayudarte hoy? (Escribe 'adiós' para salir)")

while(flag == True):
    user_response = input()
    user_response = user_response.lower()
    if(user_response != 'adiós'):
        if(user_response == 'gracias' or user_response == 'muchas gracias'):
            flag = False
            print("Chatbot: ¡De nada! ¿Hay algo más en lo que te pueda ayudar?")
        else:
            if saludos(user_response) is not None:
                print("Chatbot: " + saludos(user_response))
            else:
                print("Chatbot: ", end="")
                print(response(user_response))
    else:
        flag = False
        print("Chatbot: ¡Adiós! Que tengas un buen día.")

Chatbot: ¡Hola! Soy tu asistente virtual. ¿En qué puedo ayudarte hoy? (Escribe 'adiós' para salir)
Chatbot: Hola, ¿Qué tal?


c:\Users\dg_su\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Chatbot: Lo siento, no te he entendido. Si no puedo responder a lo que buscas, ponte en contacto con soporte@empresa.com.
Chatbot: Lo siento, no te he entendido. Si no puedo responder a lo que buscas, ponte en contacto con soporte@empresa.com.
Chatbot: Hola, encantado de hablar contigo
Chatbot: ¿te gustaría saber más sobre alguno de estos servicios?
Chatbot: desarrollamos todo tipo de páginas web, desde sitios web corporativos y tiendas en línea hasta blogs y portafolios personales.
Chatbot: la marca de tu empresa se va a volver digital con nosotros en solo 3 dias
Chatbot: Lo siento, no te he entendido. Si no puedo responder a lo que buscas, ponte en contacto con soporte@empresa.com.
Chatbot: Lo siento, no te he entendido. Si no puedo responder a lo que buscas, ponte en contacto con soporte@empresa.com.
Chatbot: ¡Adiós! Que tengas un buen día.
